# Data Preprocessing of the Covid-19 Dataset

In [ ]:
!pip install -r requirements.txt

In [ ]:
import datetime as dt
from datetime import timedelta
from time import time

import pandas as pd
from missingpy import MissForest
# noinspection PyUnresolvedReferences
from nltk.corpus import stopwords

In [ ]:
# Download the owid_covid-data CSV from https://ourworldindata.org/coronavirus-source-data and place it in
# the data folder
covid_19_raw = pd.read_csv("data/owid-covid-data.csv", encoding="utf-8")

## Column Selection

In [ ]:
col_selection = ["location"
    , "date"
    , "total_cases"
    , "new_cases"
    , "new_cases_smoothed"
    , "new_deaths_smoothed"
    , "reproduction_rate"
    , "icu_patients"
    , "hosp_patients"
    , "new_tests"
    , "new_tests_smoothed"
    , "tests_per_case"
    , "positive_rate"
    , "stringency_index"
    , "population"
    , "population_density"
    , "median_age"
    , "aged_65_older"
    , "aged_70_older"
    , "gdp_per_capita"
    , "extreme_poverty"
    , "cardiovasc_death_rate"
    , "diabetes_prevalence"
    , "female_smokers"
    , "male_smokers"
    , "handwashing_facilities"
    , "hospital_beds_per_thousand"
    , "life_expectancy"
    , "human_development_index"]

# Perform Selection of Columns
covid_19_column_selection = covid_19_raw[col_selection]
covid_19_column_selection.to_excel("data/owi-covid-data_cols_filtered.xlsx")

## Drop Columns with more than 50% missing values

In [ ]:
eu_countries = ["Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia",
                "Finland", "France", "Germany", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Luxembourg",
                "Lithuania", "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia",
                "Spain", "Sweden", "United Kingdom"]
# drop columns with more than 50% missing values and not in EU country
covid_19_column_selection = covid_19_column_selection[covid_19_column_selection["location"].isin(eu_countries)]
covid_19_column_selection = covid_19_column_selection.dropna(
    thresh=len(covid_19_column_selection) * 0.5, axis=1)
# convert column to integer
covid_19_column_selection["date"] = pd.to_datetime(covid_19_column_selection["date"])
covid_19_column_selection['date'] = covid_19_column_selection['date'].map(dt.datetime.toordinal)
# Pivot the Location Column
transformed_col = pd.get_dummies(covid_19_column_selection)
t1 = time()
percent_missing = transformed_col.isnull().sum() * 100 / len(transformed_col)
missing_value_df = pd.DataFrame({'column_name': transformed_col.columns,
                                 'percent_missing': percent_missing})
missing_value_df.to_excel("data/missing_value_information.xlsx")

## MissForest missing value imputation

In [ ]:
# Make an instance and perform the imputation
imputer = MissForest(verbose=1, max_iter=15, n_jobs=-1)
# Impute Missing Values
covid_19_values_imputed = pd.DataFrame(imputer.fit_transform(transformed_col), columns=transformed_col.columns.tolist())
t2 = time()
covid_19_values_imputed.to_excel("data/owi-covid-faulty_values_imputed.xlsx")

## Deletion of Faulty Imputed Data

In [ ]:
# Delete Rows with Faulty Imputed data
covid_19_values_imputed = covid_19_values_imputed[covid_19_values_imputed["extreme_poverty"] < 1]
covid_19_values_imputed = covid_19_values_imputed[covid_19_values_imputed["new_cases"] >= 0]
covid_19_values_imputed = covid_19_values_imputed[covid_19_values_imputed["new_cases_smoothed"] >= 0]
print()
print(f"Execution Time for Imputation {timedelta(seconds=(t2 - t1))}")

# Export the Files for Analysis
covid_19_values_imputed.to_excel("data/owi-covid-values_imputed.xlsx")
covid_19_values_imputed.to_json("data/owi-covid-values_imputed.json")

## (Optional) perform steps for whole dataset (non-EU countries incl.)

In [ ]:
# Now for every country on the world
t1 = time()
# Transform columns with all countries selected
transformed_col = pd.get_dummies(covid_19_column_selection[~covid_19_column_selection["location"].isin(["World"])])
covid_19_values_imputed = pd.DataFrame(imputer.fit_transform(transformed_col), columns=transformed_col.columns.tolist())
t2 = time()
# Delete Rows with Faulty Imputed data
covid_19_values_imputed = covid_19_values_imputed[covid_19_values_imputed["extreme_poverty"] < 1]
covid_19_values_imputed = covid_19_values_imputed[covid_19_values_imputed["new_cases"] >= 0]
covid_19_values_imputed = covid_19_values_imputed[covid_19_values_imputed["new_cases_smoothed"] >= 0]
print()
print(f"Execution Time for Imputation {timedelta(seconds=(t2 - t1))}")
# Export the Files for Analysis
covid_19_values_imputed.to_excel("data/owi-covid-values_imputed_all_countries.xlsx")
covid_19_values_imputed.to_json("data/owi-covid-values_imputed_all_countries.json")